## 프로젝트 기반 빅데이터 서비스 솔루션 개발 전문 과정

#### 교과목명 : 머신러닝응용
- 평가일 : 03.28
- 성명 :
- 점수 :

문제 : LMEMBERS의 상품구매데이터를 이용하여 개인맞춤 상품 추천솔루션을 구축 후 다양한 활용 방안을 시현하세요. 

[개요]
1. 개인 맞춤 상품 추천 솔루션 구축
- 추천하려는 target 고객과 카테고리를 정의

- '평점'으로 사용될 구매지수를 정의
    - 상품별 단가 가중치: (상품별 단가) / (단가평균)
        - 일반적으로 단가가 높으면 덜 구매함
    - 가중치가 적용된 상품별 구매건수: (상품별 건수) * (상품별 단가 가중치)
        - 단가가 낮고 건수가 많은것은 조금 줄여주고, 단가가 높고 건수가 적은것은 조금 늘려주는 방식
    - 구매지수: (가중치가 적용된 상품구매건수) / (고객별 구매건수합)
        - 상한선과 하한선이 존재하게 된다

- surprise api를 사용하기 위한 데이터셋 구성 사용자 함수

- surprise api를 이용한 다양한 모델의 구성(knnbaseline_item, svd 모델만 시현하였음)
    - 컨텐츠 기반 추천시스템은 진부함이 단점이므로 개인맞춤 상품 추천을 위해 협업필터링 위주로 하게 되었음.

- 모델 평가 지수들을 비교하여 적절한 모델 선정

- 선정된 모델로 target 고객의 상품별 구매지수를 예측

- 예측된 구매지수를 기준으로 정렬하여 추천상품 선별



2. 활용방안 제시 - 추천 시스템들의 단점을 보완
- 감소고객 예측 프로젝트의 솔루션으로 활용 (시현완료)
    - 감소고객 예측 프로젝트에서 나온 "유의고객"들이 특성별로 군집화 되어 있음
    - 해당 군집 안에서 고객번호와 취약한 상품분류를 입력하여 해당 고객에게 알맞은 상품 분류를 추천하여 매출 감소에 대한 솔루션을 제시
- 월별 인기상품과 결합 (시현완료)
    - 판매되는 상품들은 상시판매되는 상품들과 계절적 특성이 들어간 상품이 있음
    - 작년 해당월의 인기상품을 추려내어(구매지수 활용) 위에서 구한 추천상품과 결합
    - 데이터 특성상 '소분류'로 추천하기 때문에, 신상품 미반영에 대한 우려는 없음
- 여러 모델들의 결합 - hybrid recommendation system (간접시현완료)
    - 시현에서 협업시스템 위주로 하였지만, 아이템 기반 추천의 장점을 무시할 수는 없는 것처럼 다양한 모델은 각각 장점과 단점이 있음
    - 사용처와 데이터에 따라 적절하게 가중치를 두고 결합하여 hybrid 추천시스템을 구현할 수 있음.

### 개인화 추천 솔루션 구축


In [ ]:
# 타겟고객과 카테고리의 정의
target_id = "'00001'"
prodcl = "'식료품'"

In [6]:
def get_scode(prodcl=None):
    import cx_Oracle
    import pandas as pd
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    cursor.execute(f"SELECT 상품분류,소분류코드,소분류명 FROM PRODCL WHERE 상품분류 = {prodcl} ORDER BY 소분류코드")
    x = cursor.fetchall()
    col = ['상품분류','iid','소분류명']
    df = pd.DataFrame(x,columns=col)
    return df


In [4]:
# 입력받은 상품분류에 대해 소분류코드와 소분류명을 가져오는 사용자 함수
prodcl = "'식료품'"
prodcl_scode = get_scode(prodcl)
prodcl_scode

,상품분류,iid,소분류명
0,식료품,A010201,우육
1,식료품,A010202,돈육
2,식료품,A010203,계육
3,식료품,A010204,갈비세트
4,식료품,A010205,정육세트
...,...,...,...
2016,식료품,D040219,녹차/홍차
2017,식료품,D040220,허브/꽃차
2018,식료품,D040221,전통/한방차
2019,식료품,D040222,코코아


In [53]:
# 데이터베이스에서 필요한 데이터세트를 가져오는 사용자함수
# 프로젝트에서 마지막 분기를 제외한 데이터를 사용하므로, 15년 10월 1일까지 데이터만을 가지고 훈련하게 된다.

def get_data(uid = None,prodcl=None,date = 20151001):
    import cx_Oracle
    import pandas as pd
    import numpy as np
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    
    # 타겟으로 들어온 고객의 전 상품에 대한 정보가 필요.
    # purprod와 prodcl로 사용하면 outer조인이 원하는대로 작동하지 않아 각각 view로 만들어주고 outer 조인
    cursor.execute(f"CREATE VIEW TEMP1 AS SELECT 상품분류, 소분류코드 FROM PRODCL WHERE 상품분류 = {prodcl}")
    cursor.execute(f"CREATE VIEW TEMP2 AS SELECT 고객번호, PURPROD.소분류코드, COUNT(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 상품별구매건수, SUM(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 상품별구매총액 FROM PURPROD, PRODCL WHERE PURPROD.소분류코드 = PRODCL.소분류코드 AND 상품분류 = {prodcl} AND 고객번호 = {uid} GROUP BY PURPROD.고객번호, PURPROD.소분류코드 ORDER BY PURPROD.고객번호, PURPROD.소분류코드")
    cursor.execute(f"SELECT NVL(고객번호,{uid}) 고객번호, TEMP1.소분류코드, 상품별구매건수, 상품별구매총액 FROM TEMP2 RIGHT OUTER JOIN TEMP1 ON TEMP2.소분류코드 = TEMP1.소분류코드 ORDER BY 고객번호, TEMP2.소분류코드")
    t = cursor.fetchall()
    col = ['고객번호','소분류코드','상품별구매건수','상품별구매총액']
    df1 = pd.DataFrame(t,columns=col)
    df1['상품별구매건수'] = df1['상품별구매건수'].replace(0,np.nan)

    # 타겟고객을 제외한 다른 고객들의 데이터를 가져온다. 이것은 null이 없도록 한다.
    cursor.execute(f"SELECT 고객번호, PURPROD.소분류코드, COUNT(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 상품별구매건수, SUM(CASE WHEN 구매일자 < {date} THEN 구매금액 END) 상품별구매총액 FROM PURPROD,PRODCL WHERE PURPROD.소분류코드 = PRODCL.소분류코드 AND 상품분류 = {prodcl} AND 고객번호 <> {uid} GROUP BY PURPROD.고객번호, PURPROD.소분류코드 ORDER BY PURPROD.고객번호, PURPROD.소분류코드")
    x = cursor.fetchall()
    col = ['고객번호','소분류코드','상품별구매건수','상품별구매총액']
    df = pd.DataFrame(x,columns=col)
    df = df.dropna()
    # 임시로 만든 뷰는 삭제한다.
    cursor.execute(f"DROP VIEW TEMP1")
    cursor.execute(f"DROP VIEW TEMP2")

    # 구매지수로 정의한 것을 만들기 위한 과정. 단가 > 단가가중치 > 가중치가 적용된 건수
    df = pd.concat([df1,df],axis=0).reset_index().drop('index',axis=1)
    df['상품별단가'] = df['상품별구매총액'] / df['상품별구매건수'] # 단가 계산
    meanprice = df['상품별단가'].mean()
    df['단가가중치'] = df['상품별단가'].apply(lambda x: x/meanprice) # 단가를 단가 평균으로 나눠주면 가중치가 나온다
    df['상품별구매건수'] = df['상품별구매건수'] * df['단가가중치'] # 건수에 가중치를 곱해준 것으로 새로운 건수를 만든다
    df.drop(['상품별단가','단가가중치'],axis=1,inplace=True)
    # 가중치가 적용된 건수를 이용하여 고객마다의 비율을 산출
    tdf = pd.DataFrame(df.groupby('고객번호')['상품별구매건수'].sum()) # 새로운건수의 총합을 구함
    tdf = tdf.reset_index()
    mdf = pd.merge(df,tdf,on='고객번호')
    mdf.columns =['고객번호','소분류코드','상품별구매건수','상품별구매총액','총구매건수']
    
    # 최대값은 10으로 준다.
    mdf['상품별구매비율'] = mdf.상품별구매건수 * 10 / mdf.총구매건수
    
    mdf = mdf.drop(['상품별구매건수','상품별구매총액','총구매건수'],axis=1)
    return mdf

In [54]:
# 1분 소요되었음.
target_id = "'00001'"
dd = get_data(uid=target_id,prodcl=prodcl)
dd

,고객번호,소분류코드,상품별구매비율
0,00001,A070101,2.420852
1,00001,A070102,3.598999
2,00001,A070301,0.058190
3,00001,A070601,1.613901
4,00001,A070602,1.727771
...,...,...,...
30190,19374,A070603,6.243719
30191,19374,A070607,3.756281
30192,19379,D050302,10.000000
30193,19380,D050302,4.651163


In [250]:
# 구매지수 null값은 타겟고객에만 있음. 
dd[(dd.고객번호 == '00001')&(dd.상품별구매비율.isnull())]

,고객번호,소분류코드,상품별구매비율
106,00001,C040406,NaN
124,00001,C080303,NaN
127,00001,C100203,NaN
136,00001,C150102,NaN
138,00001,C150302,NaN
...,...,...,...
2016,00001,C150602,NaN
2017,00001,C150603,NaN
2018,00001,C150604,NaN
2019,00001,C150701,NaN


In [142]:
import pandas as pd
import numpy as np

from surprise import SVD, SVDpp, Reader
from surprise import Dataset
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from surprise import dump

In [236]:
# 데이터 구성 밑단계
# 훈련데이터와 타겟데이터를 분리합니다. null값은 앞서 얘기했듯 target데이터에만 있다.
# target에 대한 데이터는 추천에 용이하도록 둘 다 포함되도록 한다.

data = dd[dd.상품별구매비율.isnull() == False]
target_data = dd[(dd.고객번호 == '00001')]

# 데이터셋을 train과 test로 나누기 위해 섞어준다.
data = data.sample(frac=1,random_state=0)
cutoff = int(.75 * len(data))
train_data = data[:cutoff]
test_data = data[cutoff:]
test_data.head()

,고객번호,소분류코드,상품별구매비율
3803965,17589,C050308,0.005444
4104224,18956,B120402,0.529544
479193,02480,B520301,0.011311
2227177,10553,C110101,0.000934
818328,04053,B380304,0.008614


In [235]:
target_data

,고객번호,소분류코드,상품별구매비율
0,00001,A010201,0.028628
1,00001,A010202,0.064948
2,00001,A010207,0.060119
3,00001,A010301,0.018322
4,00001,A010302,0.155317
...,...,...,...
2016,00001,C150602,NaN
2017,00001,C150603,NaN
2018,00001,C150604,NaN
2019,00001,C150701,NaN


In [8]:
# surprise api 이용을 위해 데이터셋 구성하는 사용자 함수.
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe, target_dataframe):
    # 최고점은 10점이므로 다음과 같이 구성하였다.
    reader = Reader(rating_scale=(0.001, 10))
    trainset = Dataset.load_from_df(training_dataframe[['고객번호', '소분류코드', '상품별구매비율']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['고객번호', '소분류코드', '상품별구매비율']], reader)
    targetset = Dataset.load_from_df(target_dataframe[['고객번호', '소분류코드', '상품별구매비율']], reader)
    
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)

    # 추가적으로, 마지막 예측을 위한 targetset 역시 테스트 데이터로 구성한다.
    targetset = targetset.construct_testset(targetset.raw_ratings)
    
    return trainset, testset, targetset

In [ ]:
trainset, testset, targetset = convert_traintest_dataframe_forsurprise(train_data,test_data,target_data)

In [239]:
# Knn-baseline 모델 
# 7분 40초 소요
# baseline이 포함되면 깐깐한 사람은 점수를 짜게 주고, 후한 사람은 후하게 주는 것을 고려해준다.
# Reference: https://danthetech.netlify.app/DataScience/how-does-recommendation-algorithms-work-using-surpriselib
# https://surprise.readthedocs.io/en/stable/knn_inspired.html

sim_options = {'name': 'cosine',
               'user_based': False  # content(item)-based similarity
               }

# als와 sgd 방식이 있는데, 책에서 공부한 sgd 방식으로 선언.
bsl_options = {'method': 'sgd'}
# 참고: ALS방식
# 하나의 latent를 고정하고 진행시키는 것이다.
# 사용자 행렬을 초기화하고 처음에는 아이템 행렬을 고정하고 사용자 행렬을 최적화한다.
# 그 다음에는 사용자 행렬을 고정하고 아이템 행렬을 최적화한다.

# KNNBaseline 객체 선언. k = 1000, min_k = 50등과 같은 파라미터도 지정 가능하지만, 비교를 위해 주지 않았다.
knnbaseline_algo_i = KNNBaseline(sim_options=sim_options,bsl_options=bsl_options)

knnbaseline_algo_i.fit(trainset)
knnbaseline_i_pred = knnbaseline_algo_i.test(testset)

# dump 모듈을 사용해 알고리즘과 예측한 값들을 저장할 수 있다.
# predictions는 따로 저장하지 않았다.
file_name = './dataset/KnnBaseline_model_i'
dump.dump(file_name, algo=knnbaseline_algo_i)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(knnbaseline_i_pred)
accuracy.mae(knnbaseline_i_pred)
print("Done!")

Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.1413
MAE:  0.0428
Done!


In [307]:
# accuracy 모듈을 통해 평가 지표를 산출한다.
accuracy.rmse(knnbaseline_i_pred)
accuracy.mae(knnbaseline_i_pred)

RMSE: 0.1413
MAE:  0.0428


0.04278373802909311

In [323]:
temp = pd.DataFrame(knnbaseline_i_pred)
temp[temp.uid == '18956'].sort_values('est',ascending=False).head()

,uid,iid,r_ui,est,details
1,18956,B120402,0.529544,0.340641,"{'actual_k': 40, 'was_impossible': False}"
753056,18956,B040101,0.021609,0.259188,"{'actual_k': 40, 'was_impossible': False}"
132645,18956,B480301,0.096686,0.123504,"{'actual_k': 40, 'was_impossible': False}"
589750,18956,A010624,0.015517,0.109108,"{'actual_k': 40, 'was_impossible': False}"
224739,18956,B130501,0.094690,0.107535,"{'actual_k': 40, 'was_impossible': False}"


In [260]:
# 이제 따로 빼둔 타겟 고객의 데이터를 예측하고 예측지수로 정렬하여 추천상품을 고른다.
knnbaseline_i_target_pred = knnbaseline_algo_i.test(targetset)
knn = pd.DataFrame(knnbaseline_i_target_pred)
knn = knn.drop('details',axis=1).sort_values('est',ascending=False)
# 앞에서 구해둔 prodcl_scode에서 소분류명을 가져온다.
knn = knn.merge(prodcl_scode.drop('상품분류',axis=1),on='iid')
knn

,uid,iid,r_ui,est,소분류명
0,00001,A010801,2.850385,1.677605,건강식품(홍삼)
1,00001,A010205,NaN,1.516918,정육세트
2,00001,A010204,NaN,0.925746,갈비세트
3,00001,A010803,NaN,0.751559,건강식품(비타민)
4,00001,D030205,NaN,0.747992,다이어트보조식품
...,...,...,...,...,...
2016,00001,B380401,NaN,0.010000,두부
2017,00001,B380307,NaN,0.010000,씨제이콩나물
2018,00001,B380306,NaN,0.010000,대상종가집콩나물
2019,00001,B380305,NaN,0.010000,풀무원콩나물


In [261]:
knn[:20]

,uid,iid,r_ui,est,소분류명
0,00001,A010801,2.850385,1.677605,건강식품(홍삼)
1,00001,A010205,NaN,1.516918,정육세트
2,00001,A010204,NaN,0.925746,갈비세트
3,00001,A010803,NaN,0.751559,건강식품(비타민)
4,00001,D030205,NaN,0.747992,다이어트보조식품
5,00001,A010403,0.157045,0.741427,유기농채소
6,00001,A010515,NaN,0.731374,선어세트
7,00001,A010401,1.101450,0.706125,청과
8,00001,A010409,NaN,0.677453,청과세트
9,00001,A010802,NaN,0.673361,건강식품


In [93]:
# # Knn-baseline 모델
# # 데이터셋 수정하기 전에 실행하는데 113분이 소요되었다. 이 모델은 시간적 여유가 있을 때 사용해보려 한다.
# # User Based Filterting(UBF)
# ## 참조: https://surprise.readthedocs.io/en/stable/knn_inspired.html

# sim_options = {'name': 'cosine',
#                'user_based': True  # user-based similarity
#                }
# knnbaseline_algo_u = KNNBaseline(sim_options=sim_options)

# knnbaseline_algo_u.fit(trainset)
# knnbaseline_u_pred = knnbaseline_algo_u.test(testset)

# file_name = './dataset/KnnBaseline_model_u'
# dump.dump(file_name, algo=knnbaseline_algo_u)
# # _, loaded_algo = dump.load(file_name)

# accuracy.rmse(knnbaseline_u_pred)
# accuracy.mae(knnbaseline_u_pred)
# print("Done!")

Estimating biases using als...
Computing the cosine similarity matrix...


C:\Users\chonh\anaconda3\envs\cakd5\lib\site-packages\surprise\prediction_algorithms\algo_base.py:249: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
RMSE: 0.1145
MAE:  0.0107
Done!


In [245]:
# SVD
# 이 모델은 fit 과정을 반복함에 있어 SGD 방식으로 한다고 한다.
## 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#matrix-factorization-based-algorithms

svd_algo = SVD()

svd_algo.fit(trainset)
svd_pred = svd_algo.test(testset)

file_name = './dataset/svd_model'
dump.dump(file_name,algo=svd_algo)
# _, loaded_algo = dump.load(file_name)

accuracy.rmse(svd_pred)
accuracy.mae(svd_pred)
print("Done!")

RMSE: 0.1432
MAE:  0.0438
Done!


In [258]:
# svd 모델로 target을 예측해보고, 소분류명을 불러온다.
svd_target_pred = svd_algo.test(targetset)
svd = pd.DataFrame(svd_target_pred)
svd = svd.drop('details',axis=1).sort_values('est',ascending=False)
svd = svd.merge(prodcl_scode.drop('상품분류',axis=1),on='iid')
svd

,uid,iid,r_ui,est,소분류명
0,00001,A010801,2.850385,1.755605,건강식품(홍삼)
1,00001,A010205,NaN,1.148183,정육세트
2,00001,A010204,NaN,0.952929,갈비세트
3,00001,A010403,0.157045,0.831379,유기농채소
4,00001,A010803,NaN,0.777536,건강식품(비타민)
...,...,...,...,...,...
2016,00001,B430403,NaN,0.010000,냉장간편식
2017,00001,B430301,NaN,0.010000,햄
2018,00001,B430202,NaN,0.010000,냉장떡
2019,00001,B430105,NaN,0.010000,김밥재료묶음


In [259]:
# svd 기반 추천리스트
svd[:20]

,uid,iid,r_ui,est,소분류명
0,00001,A010801,2.850385,1.755605,건강식품(홍삼)
1,00001,A010205,NaN,1.148183,정육세트
2,00001,A010204,NaN,0.952929,갈비세트
3,00001,A010403,0.157045,0.831379,유기농채소
4,00001,A010803,NaN,0.777536,건강식품(비타민)
5,00001,A010515,NaN,0.745861,선어세트
6,00001,A010401,1.101450,0.737970,청과
7,00001,A010802,NaN,0.736227,건강식품
8,00001,A010901,NaN,0.693022,주류
9,00001,D030205,NaN,0.685933,다이어트보조식품


In [ ]:
# # SVD++
# # 이 모델 역시 굉장히 오래걸리므로(70분에 중단하였음) 이번엔 사용하지 않겠다.
# ## svd++ model : taking into account implicit ratings
# ## 참고: https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVDpp

# svdpp_algo = SVDpp()

# svdpp_algo.fit(trainset)
# svdpp_pred = svdpp_algo.test(testset)

# file_name = './dataset/svdpp_model'
# dump.dump(file_name, algo=svdpp_algo)
# # _, loaded_algo = dump.load(file_name)

# accuracy.rmse(svdpp_pred)
# accuracy.mae(svdpp_pred)
# print("Done!")

### 만들어진 솔루션의 활용방안


1. 구매감소고객 예측 프로젝트의 솔루션으로 활용

In [2]:
import pandas as pd
import numpy as np

from surprise import SVD, SVDpp, Reader
from surprise import Dataset
from surprise.prediction_algorithms import KNNBaseline
from surprise import accuracy
from surprise import dump

In [9]:
# ex) 타겟 중 한 군집의 고객들은 가전제품 영역의 구매가 떨어졌던 것이 매출감소의 주 원인이었다고 한다.
# 이 고객들에게 가전제품 영역의 제품들을 추천해야 할 것이다.
# 같은 군집이어도 고객마다의 취향이 다른 것이 고려되어야 하므로 위에서 구축한 시스템이 적합하다.
# 개개인에게 맞는 솔루션이 제시될텐데, 해당 군집의 고객이 5번고객이라고 생각하고 시현해 보겠다.

target_id = "'00005'"
prodcl = "'가전제품'"

In [10]:
prodcl_scode = get_scode(prodcl)
prodcl_scode

,상품분류,iid,소분류명
0,가전제품,A070101,국산A/V
1,가전제품,A070102,국산냉장고.세탁기
2,가전제품,A070103,수입냉장고
3,가전제품,A070104,수입세탁기
4,가전제품,A070105,수입A/V
...,...,...,...
170,가전제품,C170356,녹즙기/원액기
171,가전제품,C170357,체중계/신장계
172,가전제품,D050301,헤어드라이어
173,가전제품,D050302,헤어세팅기


In [11]:
dd = get_data(uid=target_id,prodcl=prodcl)

In [12]:
data = dd[dd.상품별구매비율.isnull() == False]
target_data = dd[(dd.고객번호 == '00005')]

# 데이터셋을 train과 test로 나누기 위해 섞어준다.
data = data.sample(frac=1,random_state=0)
cutoff = int(.75 * len(data))
train_data = data[:cutoff]
test_data = data[cutoff:]
test_data.head()

,고객번호,소분류코드,상품별구매비율
18518,10132,C170314,8.143091
8782,04331,C170314,3.233696
2461,01084,A070302,0.014489
26738,16467,C170330,2.044655
30172,19346,C170355,0.273834


In [13]:
# surprise에 맞도록 데이터를 구성
trainset, testset, targetset = convert_traintest_dataframe_forsurprise(train_data,test_data,target_data)

In [14]:
# Knn-baseline 모델 

sim_options = {'name': 'cosine',
               'user_based': False  # content(item)-based similarity
               }

bsl_options = {'method': 'sgd'}

knnbaseline_algo_i = KNNBaseline(sim_options=sim_options,bsl_options=bsl_options)

knnbaseline_algo_i.fit(trainset)
knnbaseline_i_pred = knnbaseline_algo_i.test(testset)

accuracy.rmse(knnbaseline_i_pred)
accuracy.mae(knnbaseline_i_pred)
print("Done!")

Estimating biases using sgd...
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 4.0142
MAE:  3.2103
Done!


In [15]:
# 이제 따로 빼둔 타겟 고객의 데이터를 예측하고 예측지수로 정렬하여 추천상품을 고른다.
knnbaseline_i_target_pred = knnbaseline_algo_i.test(targetset)
knn = pd.DataFrame(knnbaseline_i_target_pred)
knn = knn.drop('details',axis=1).sort_values('est',ascending=False)
# 앞에서 구해둔 prodcl_scode에서 소분류명을 가져온다.
knn = knn.merge(prodcl_scode.drop('상품분류',axis=1),on='iid')
knn[:20]

# 결과물에서 평점이 다 10점으로 통일된 것은, 해당 데이터에서 이 사람의 구매 데이터가 부족하기 때문이다.
# 원래대로 추천하였다면 다른 카테고리에서 추천하였겠지만,
# 단순히 가전제품이 문제가 되었던 고객군의 예시로 들었던 것이기에 시현만 하고 넘어가도록 한다.

,uid,iid,r_ui,est,소분류명
0,00005,A070103,10.0,10.0,수입냉장고
1,00005,A070608,NaN,10.0,에스프레소
2,00005,B410803,NaN,10.0,조리가전
3,00005,C170340,NaN,10.0,전기밥솥
4,00005,A070501,NaN,10.0,명품오디오
5,00005,C170318,NaN,10.0,선풍기
6,00005,B870401,NaN,10.0,도어락
7,00005,B390502,NaN,10.0,포터블카세트
8,00005,A070603,NaN,10.0,수입소형가전
9,00005,A070605,NaN,10.0,김치냉장고


In [16]:
# SVD

svd_algo = SVD()

svd_algo.fit(trainset)
svd_pred = svd_algo.test(testset)

accuracy.rmse(svd_pred)
accuracy.mae(svd_pred)
print("Done!")

RMSE: 3.4810
MAE:  3.0134
Done!


In [17]:
# svd 모델로 target을 예측해보고, 소분류명을 불러온다.
svd_target_pred = svd_algo.test(targetset)
svd = pd.DataFrame(svd_target_pred)
svd = svd.drop('details',axis=1).sort_values('est',ascending=False)
svd = svd.merge(prodcl_scode.drop('상품분류',axis=1),on='iid')
svd[:20]

# 위에서와는 조금 다르게 예측되었다. 이 예시가 모델별 장단점을 보여주는 예시가 될 것이다.

,uid,iid,r_ui,est,소분류명
0,00005,A070103,10.0,9.791764,수입냉장고
1,00005,B390301,NaN,8.616928,LED TV
2,00005,B850203,NaN,8.568220,전기요/장판
3,00005,B870105,NaN,8.016423,학습용스탠드
4,00005,C170324,NaN,7.370335,온수매트
5,00005,B420502,NaN,7.188535,노트북
6,00005,D060502,NaN,6.988480,이어폰/헤드폰
7,00005,B870401,NaN,6.848466,도어락
8,00005,B850108,NaN,6.837918,이미용기
9,00005,B410501,NaN,6.835113,믹서기


2. 여러 모델들의 결합 - hybrid recommendation system

In [ ]:
# 1번의 예시에서 knn 방법이 rmse가 높으므로 일반적으로 svd 방식을 채택할 것이다.
# 하지만 아이템 기반에서의 해석이 완전히 무시되고 잠재요인 방법인 svd만을 고려하는 것은 좋지 않을 수 있다.
# 상품판매에서 아이템의 특성 역시 같이 고려되어야 하고, 아마존은 그 방식을 사용하여 판매가 계속되고 있다.
# 이것을 고려하기 위하여 hybrid 모델을 사용해 보겠다
# hybrid = (0.3 * knn) + (0.7 * svd)로 가중치를 두어 살펴본다. 

In [18]:
knn.head()

,uid,iid,r_ui,est,소분류명
0,00005,A070103,10.0,10.0,수입냉장고
1,00005,A070608,NaN,10.0,에스프레소
2,00005,B410803,NaN,10.0,조리가전
3,00005,C170340,NaN,10.0,전기밥솥
4,00005,A070501,NaN,10.0,명품오디오


In [19]:
svd.head()

,uid,iid,r_ui,est,소분류명
0,00005,A070103,10.0,9.791764,수입냉장고
1,00005,B390301,NaN,8.616928,LED TV
2,00005,B850203,NaN,8.568220,전기요/장판
3,00005,B870105,NaN,8.016423,학습용스탠드
4,00005,C170324,NaN,7.370335,온수매트


In [32]:
knn_temp = knn.copy()
svd_temp = svd.copy()
knn_temp = knn_temp.sort_values('iid')
svd_temp = svd_temp.sort_values('iid')
knn_temp.head()

,uid,iid,r_ui,est,소분류명
16,00005,A070101,NaN,10.000000,국산A/V
13,00005,A070102,NaN,10.000000,국산냉장고.세탁기
0,00005,A070103,10.0,10.000000,수입냉장고
33,00005,A070104,NaN,9.090010,수입세탁기
34,00005,A070105,NaN,8.979683,수입A/V


In [33]:
svd_temp.head()

,uid,iid,r_ui,est,소분류명
160,00005,A070101,NaN,3.390301,국산A/V
34,00005,A070102,NaN,5.764183,국산냉장고.세탁기
0,00005,A070103,10.0,9.791764,수입냉장고
154,00005,A070104,NaN,3.625399,수입세탁기
131,00005,A070105,NaN,4.169819,수입A/V


In [49]:
x = svd_temp['est']
y = knn_temp['est']
svd_temp['est'] = 0.7 * x + 0.3 * y

In [51]:
hybrid = svd_temp.sort_values('est',ascending=False)
hybrid[:10]

,uid,iid,r_ui,est,소분류명
0,00005,A070103,10.0,9.938779,수입냉장고
1,00005,B390301,NaN,9.593377,LED TV
2,00005,B850203,NaN,9.579057,전기요/장판
3,00005,B870105,NaN,9.416828,학습용스탠드
4,00005,C170324,NaN,9.226878,온수매트
5,00005,B420502,NaN,9.173429,노트북
6,00005,D060502,NaN,9.114613,이어폰/헤드폰
7,00005,B870401,NaN,9.073449,도어락
8,00005,B850108,NaN,9.070348,이미용기
9,00005,B410501,NaN,9.069523,믹서기


In [55]:
knn[:10]

,uid,iid,r_ui,est,소분류명
0,00005,A070103,10.0,10.0,수입냉장고
1,00005,A070608,NaN,10.0,에스프레소
2,00005,B410803,NaN,10.0,조리가전
3,00005,C170340,NaN,10.0,전기밥솥
4,00005,A070501,NaN,10.0,명품오디오
5,00005,C170318,NaN,10.0,선풍기
6,00005,B870401,NaN,10.0,도어락
7,00005,B390502,NaN,10.0,포터블카세트
8,00005,A070603,NaN,10.0,수입소형가전
9,00005,A070605,NaN,10.0,김치냉장고


In [ ]:
# 예시로 든 고객번호의 상품 구매가 충분하지 않아 rmse를 비교할 수 없으나 (knn의 rmse*0.4) + (svd의 rmse*0.6)의 값으로
# 하이브리드 모델의 rmse를 대략적으로 알 수 있다. 이것은 그 사이값이지만, 추천 상품은 개선이 될 것이다.

3. 잠재요인 방식의 추천상품과 해당 월의 인기상품을 결합하여 추천하는 추천시스템


In [262]:
# 인기상품과 결합하기 위해 인기상품 데이터프레임을 구성한다.
# 작년 해당 월에 인기 많은 상품을 추출하기 위하여 추천하려는 달의 데이터를 다시 뽑아온다.

def past_data(prodcl=None,date1 = 20140930, date2 = 20141101):
    import cx_Oracle
    import pandas as pd
    con = cx_Oracle.connect('LM_PDB/LM@localhost:1521/xepdb1')
    cursor = con.cursor()
    cursor.execute(f"SELECT PURPROD.고객번호, PURPROD.소분류코드, COUNT(CASE WHEN 구매일자 between {date1} and {date2} THEN 구매금액 END) 상품별구매건수, SUM(CASE WHEN 구매일자 between {date1} and {date2} THEN 구매금액 END) 상품별구매총액 FROM PURPROD,PRODCL WHERE PURPROD.소분류코드 = PRODCL.소분류코드 AND 상품분류 = {prodcl} GROUP BY PURPROD.고객번호, PURPROD.소분류코드 ORDER BY PURPROD.고객번호, PURPROD.소분류코드")
    x = cursor.fetchall()
    col = ['고객번호','소분류코드','상품별구매건수','상품별구매총액']
    df = pd.DataFrame(x,columns=col)
    df = df.dropna()

    df['상품별단가'] = df['상품별구매총액'] / df['상품별구매건수'] # 단가 계산
    meanprice = df['상품별단가'].mean()
    df['단가가중치'] = df['상품별단가'].apply(lambda x: x/meanprice) # 단가를 단가 평균으로 나눠주면 가중치가 나온다
    df['상품별구매건수'] = df['상품별구매건수'] * df['단가가중치'] # 건수에 가중치를 곱해준 것으로 새로운 건수를 만든다
    df.drop(['상품별단가','단가가중치'],axis=1,inplace=True)

    tdf = pd.DataFrame(df.groupby('고객번호')['상품별구매건수'].sum()) # 새로운건수의 총합을 구함
    tdf = tdf.reset_index()
    mdf = pd.merge(df,tdf,on='고객번호')
    mdf.columns =['고객번호','소분류코드','상품별구매건수','상품별구매총액','총구매건수']
    mdf['상품별구매비율'] = mdf.상품별구매건수 * 10 / mdf.총구매건수
    
    mdf = mdf.drop(['상품별구매건수','상품별구매총액','총구매건수'],axis=1)
    return mdf

past = past_data(prodcl)

In [325]:
# 위 데이터셋 구성하는 사용자 함수에 타겟 고객의 전체 상품 리스트가 제외되어 있어서 다시 포함시켜준다.
ad = dd[dd['고객번호']=='00001']
item_list = ad.소분류코드.tolist()
ndf = past[past.소분류코드.isin(item_list)]

# 인기상품의 기준은 구매비율의 평균이 높은 것으로 한다.
pop = ndf.groupby('소분류코드')['상품별구매비율'].mean().sort_values(ascending=False)
pop = pd.DataFrame(pop).reset_index()
# 아래에서 사용하기 위해 소분류코드의 열이름을 iid로 맞춰준다.
pop.columns = ['iid','r_ui']
pop = pop.merge(prodcl_scode.drop('상품분류',axis=1),on='iid')
pop

,iid,r_ui,소분류명
0,A010409,9.584293,청과세트
1,D030201,4.716182,종합영양제
2,A010801,4.244271,건강식품(홍삼)
3,D030205,3.769805,다이어트보조식품
4,B750501,3.546231,건강기능성선물세트
...,...,...,...
1360,C030108,0.033992,쑥갓
1361,C030808,0.033753,파래
1362,C150503,0.030993,풍선껌
1363,C120601,0.014012,공병/공박스


In [301]:
# 타겟 고객의 취향이 고려된 추천리스트와 해당 월의 인기상품 추천리스트를 합치는 작업
# 인기상품과 곂치는 것은 1.5의 가중치를 주었다.

svd2 = svd.copy()
for i in pop[:20].iid.tolist():
    svd2.loc[svd2.iid == i,'est'] = svd2.loc[svd2.iid == i,'est'] * 1.5
result = svd2.sort_values('est',ascending=False)[:10]
result

# 밑의 리스트와 비교해 보자면 개인 특성을 베이스로 두되, 계절적 특성이나 인기상품에 관한 것이 더 반영되었음을 알 수 있다.

,uid,iid,r_ui,est,소분류명
0,00001,A010801,2.850385,2.633408,건강식품(홍삼)
4,00001,A010803,NaN,1.166304,건강식품(비타민)
1,00001,A010205,NaN,1.148183,정육세트
7,00001,A010802,NaN,1.104340,건강식품
9,00001,D030205,NaN,1.028899,다이어트보조식품
2,00001,A010204,NaN,0.952929,갈비세트
12,00001,A010409,NaN,0.877651,청과세트
3,00001,A010403,0.157045,0.831379,유기농채소
20,00001,D030201,NaN,0.767606,종합영양제
5,00001,A010515,NaN,0.745861,선어세트


In [292]:
# 원래의 리스트
svd[:10]

,uid,iid,r_ui,est,소분류명
0,00001,A010801,2.850385,1.755605,건강식품(홍삼)
1,00001,A010205,NaN,1.148183,정육세트
2,00001,A010204,NaN,0.952929,갈비세트
3,00001,A010403,0.157045,0.831379,유기농채소
4,00001,A010803,NaN,0.777536,건강식품(비타민)
5,00001,A010515,NaN,0.745861,선어세트
6,00001,A010401,1.101450,0.737970,청과
7,00001,A010802,NaN,0.736227,건강식품
8,00001,A010901,NaN,0.693022,주류
9,00001,D030205,NaN,0.685933,다이어트보조식품
